In [1]:
import pandas as pd
import numpy as np

# 可視化用のライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import neologdn
import MeCab

import re

from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
# # プログレスバーのラベル設定
tqdm_notebook.pandas(desc="progress: ")

In [2]:
df = pd.read_pickle('posi_nega_alldata.pkl')
df.head()

,text,posi_nega
0,案件個別の技術相談乗ってます！乗ります,1
1,勉強とお仕事が有機的に繋がっていて面白い。,1
2,次々とテクニックを教えていただき、ためになりました,1
3,お肉がキレイに焼けたことに小さなしあわせを感じた,1
4,これが私の最適化,1


In [3]:
text_df = df[['text']]
text_df.head()

,text
0,案件個別の技術相談乗ってます！乗ります
1,勉強とお仕事が有機的に繋がっていて面白い。
2,次々とテクニックを教えていただき、ためになりました
3,お肉がキレイに焼けたことに小さなしあわせを感じた
4,これが私の最適化


## 形態素解析結果をarray形式で出力する関数を定義
後にarrayをデータフレーム形式に変換するための準備

形態素解析をする関数を全文章に適応させる際に返り値がarrayになる関数を使った時の方が処理スピードが速いため

In [4]:
def get_wakati_text_array(text):
    # 【neologdnで文字表現の正規】
    text_normalization = neologdn.normalize(text)
    
    # 【【形態素解析】】
    # 【辞書設定】
    #　開発環境がwindowsの場合：'-Ochasen -d C:\mecab-ipadic-neologd'
    # 開発環境がMacの場合:'-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd'
    neologd_tagger = MeCab.Tagger('-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
    # 【形態素解析を実行、それをスペースあり改行なしの文字列にする】
    # neologd_tagger.parse(text)で下記のように各単語の原形、品詞などが1行に連続して表示される
    # [表示\tヒョウジ\t表示\t名詞-サ変接続\t\t\nさ\t　.....\nEOS\n']
    # 原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される
    #　neologd_tagger.parseで形態素解析した後、形態素解析結果を「'\n'→'\t'」に変換することで単にスペースがある改行なしの１つの文字列になる
    # この文字列に対して「'\t'」でsplitすることで「単語」「その読み方」「その品詞」... が1つ1つのの要素になったリスト型になる
    # このリストの最後の２つの要素は「EOS」と「(空白文字)」であり、形態素解析結果として使用しないためこれら２つを含まないように抽出する→[:-2]
    wakati_1d_text = neologd_tagger.parse(text_normalization).replace('\n', '\t').split('\t')[:-2]
    
    # 【【形態素解析結果をデータフレームに格納する準備】】
    # この時点でwakati_1d_textは1次元のリストであり、このままの形式でpd.DataFrame関数の引数に渡してしまうと１列のデータフレームになってしまう
    # そこでnumpy.array関数とreshape関数を使い下記のような[n行（単語数分）６列(単語、読み方、原形、品詞、活用形、活用型)」のn次元配列リストを作成
    # ['原形', 'ゲンケイ', '原形', '名詞-一般', '', ''],
    # ['、', '、', '、', '記号-読点', '', ''],
    # この[n行（単語数分）６列(単語、読み方、原形、品詞、活用形、活用型)」のn次元配列リストをpd.DataFrame関数の引数に渡す
    wakati_array = np.array([wakati_1d_text]).reshape(-1, 6)
    return wakati_array

## 全文章に対し形態素解析結果をarray形式で出力する関数を実行

In [5]:
# [内容]カラムの全行に対してget_wakati_text_array関数を実行
# 行毎に形態素解析結果が['原形', 'ゲンケイ', '原形', '名詞-一般', '', '']　といった形式で格納される
# 今回apply関数をSeries型（text_df['text']）に対して使用するためaxis=1を引数に与えない
wakati_text_array = text_df['text'].progress_apply(get_wakati_text_array)

In [6]:
# 確認
wakati_text_array

0     [[案件, アンケン, 案件, 名詞-一般, , ], [個別, コベツ, 個別, 名詞-形...
1     [[勉強, ベンキョウ, 勉強, 名詞-サ変接続, , ], [と, ト, と, 助詞-並立...
2     [[次々, ツギツギ, 次々, 副詞-助詞類接続, , ], [と, ト, と, 助詞-副詞...
3     [[お肉, オニク, お肉。, 名詞-固有名詞-一般, , ], [が, ガ, が, 助詞-...
4     [[これ, コレ, これ, 名詞-代名詞-一般, , ], [が, ガ, が, 助詞-格助詞...
5     [[データ, データ, データ, 名詞-一般, , ], [の, ノ, の, 助詞-連体化,...
6     [[PC, ピーシー, PC, 名詞-固有名詞-一般, , ], [復帰, フッキ, 復帰,...
7     [[久々, ヒサビサ, 久々, 名詞-一般, , ], [に, ニ, に, 助詞-格助詞-一...
8     [[おでん, オデン, おでん, 名詞-一般, , ], [が, ガ, が, 助詞-格助詞-...
9     [[昼飯, ヒルメシ, 昼飯, 名詞-一般, , ], [に, ニ, に, 助詞-格助詞-一...
10    [[10歳, ジュッサイ, 10歳, 名詞-固有名詞-一般, , ], [以上, イジョウ,...
11    [[2週, ニシュウ, 2週, 名詞-固有名詞-一般, , ], [連続, レンゾク, 連続...
12    [[ご, ゴ, ご, 接頭詞-名詞接続, , ], [参加, サンカ, 参加, 名詞-サ変接...
13    [[無事, ブジ, 無事, 名詞-形容動詞語幹, , ], [に, ニ, に, 助詞-副詞化...
14    [[これ, コレ, これ, 名詞-代名詞-一般, , ], [も, モ, も, 助詞-係助詞...
15    [[本当にありがとうございました, ホントウニアリガトウゴザイマシタ, 本当にありがとうござ...
16    [[本, ホン, 本, 名詞-一般, , ], [を, ヲ, を, 助詞-格助詞-一般, ,...
17    [[ベイズ, ベイズ, ベイズ, 名詞-一般, , ], [統計, トウケイ, 統計

## arrayからDataFrameに変換

In [7]:
# wakati_text_arrayから１要素（リストになっている）毎取り出し、それをデータフレーム型にするリスト内包表記
# wakati_df_listには形態素解析結果を格納したデータフレームが１要素として格納される
wakati_df_list = [pd.DataFrame(wakati_text) for wakati_text in wakati_text_array]

In [8]:
# 確認
wakati_df_list[:2]

[     0     1   2          3      4      5
 0   案件  アンケン  案件      名詞-一般              
 1   個別   コベツ  個別  名詞-形容動詞語幹              
 2    の     ノ   の     助詞-連体化              
 3   技術  ギジュツ  技術      名詞-一般              
 4   相談  ソウダン  相談    名詞-サ変接続              
 5   乗っ    ノッ  乗る      動詞-自立  五段・ラ行  連用タ接続
 6    て     テ   て    助詞-接続助詞              
 7   ます    マス  ます        助動詞  特殊・マス    基本形
 8    !     !   !      記号-一般              
 9   乗り    ノリ  乗る      動詞-自立  五段・ラ行    連用形
 10  ます    マス  ます        助動詞  特殊・マス    基本形,
       0      1    2           3         4      5
 0    勉強  ベンキョウ   勉強     名詞-サ変接続                 
 1     と      ト    と     助詞-並立助詞                 
 2   お仕事   オシゴト  お仕事  名詞-固有名詞-一般                 
 3     が      ガ    が   助詞-格助詞-一般                 
 4   有機的  ユウキテキ  有機的  名詞-固有名詞-一般                 
 5     に      ニ    に   助詞-格助詞-一般                 
 6   繋がっ   ツナガッ  繋がる       動詞-自立     五段・ラ行  連用タ接続
 7     て      テ    て     助詞-接続助詞                 
 8     い      イ   いる      動詞-非自立 

## 形態素解析後の各単語(や原形、品詞等)と形態素解析前の文章の紐付け

In [9]:
# wakati_df_listには形態素解析結果を格納したデータフレームが１要素として格納されている
# しかしながら、このままだとどの単語がどの文章から形態素解析された単語であったかどうか確認することができない
# そこで、形態素解析された単語に対し元の文章を紐付けていく
# text_df['内容']:Series型　　Series型に「.values」を追加するとarrayに変換できる
# 今回の場合はtext_df['内容']列１つに対してarray変換したので１次元配列リスト
# そのリストの要素は文章１つ１つでインデックス0番目に最初の文章が格納されている
# よって、このtext_df['内容'].valuesのn番目の文章とwakati_df_listのn番目のデータフレームは紐づいていることになる
#　以上より、複数のリストの要素を同時に取得できるzip関数を使用しこれら2つを同時に取り出し紐付けるfor文を作成
#　その際、assign関数でデータフレーム（変数b）に新しいカラム[内容]を追加、そのカラムに文章（変数a）を割り当てる
array_df_tmp = []
for a, b in zip(text_df['text'].values, wakati_df_list):
    array_df_tmp.append(b.assign(内容 = a))

In [10]:
# 確認(確認しやすいように０番目の要素のみ）
array_df_tmp[0]

,0,1,2,3,4,5,内容
0,案件,アンケン,案件,名詞-一般,,,案件個別の技術相談乗ってます！乗ります
1,個別,コベツ,個別,名詞-形容動詞語幹,,,案件個別の技術相談乗ってます！乗ります
2,の,ノ,の,助詞-連体化,,,案件個別の技術相談乗ってます！乗ります
3,技術,ギジュツ,技術,名詞-一般,,,案件個別の技術相談乗ってます！乗ります
4,相談,ソウダン,相談,名詞-サ変接続,,,案件個別の技術相談乗ってます！乗ります
5,乗っ,ノッ,乗る,動詞-自立,五段・ラ行,連用タ接続,案件個別の技術相談乗ってます！乗ります
6,て,テ,て,助詞-接続助詞,,,案件個別の技術相談乗ってます！乗ります
7,ます,マス,ます,助動詞,特殊・マス,基本形,案件個別の技術相談乗ってます！乗ります
8,!,!,!,記号-一般,,,案件個別の技術相談乗ってます！乗ります
9,乗り,ノリ,乗る,動詞-自立,五段・ラ行,連用形,案件個別の技術相談乗ってます！乗ります


## リストに格納されたデータフレームをすべて連結

In [11]:
# array_df_tmpリストの中に形態素解析後の結果を格納したデータフレームが全文章分ある
# よって、このリストをpd.concat関数の引数に与えることでそれらをまとめて１度で連結できる
# axis=0 : 縦に連結
all_wakati_df = pd.concat(array_df_tmp, axis=0)

In [12]:
# concatされたか確認
all_wakati_df.head(50)

,0,1,2,3,4,5,内容
0,案件,アンケン,案件,名詞-一般,,,案件個別の技術相談乗ってます！乗ります
1,個別,コベツ,個別,名詞-形容動詞語幹,,,案件個別の技術相談乗ってます！乗ります
2,の,ノ,の,助詞-連体化,,,案件個別の技術相談乗ってます！乗ります
3,技術,ギジュツ,技術,名詞-一般,,,案件個別の技術相談乗ってます！乗ります
4,相談,ソウダン,相談,名詞-サ変接続,,,案件個別の技術相談乗ってます！乗ります
5,乗っ,ノッ,乗る,動詞-自立,五段・ラ行,連用タ接続,案件個別の技術相談乗ってます！乗ります
6,て,テ,て,助詞-接続助詞,,,案件個別の技術相談乗ってます！乗ります
7,ます,マス,ます,助動詞,特殊・マス,基本形,案件個別の技術相談乗ってます！乗ります
8,!,!,!,記号-一般,,,案件個別の技術相談乗ってます！乗ります
9,乗り,ノリ,乗る,動詞-自立,五段・ラ行,連用形,案件個別の技術相談乗ってます！乗ります


## データフレームのカラム名を変更しcsvファイルで出力

In [13]:
# カラム名を変更
all_wakati_df = all_wakati_df.rename(columns = {0:'surface', 1:'yomi', 2:'original', 3:'type', 4:'katsuyoukei', 5:'katsuyougata'})
all_wakati_df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata,内容
0,案件,アンケン,案件,名詞-一般,,,案件個別の技術相談乗ってます！乗ります
1,個別,コベツ,個別,名詞-形容動詞語幹,,,案件個別の技術相談乗ってます！乗ります
2,の,ノ,の,助詞-連体化,,,案件個別の技術相談乗ってます！乗ります
3,技術,ギジュツ,技術,名詞-一般,,,案件個別の技術相談乗ってます！乗ります
4,相談,ソウダン,相談,名詞-サ変接続,,,案件個別の技術相談乗ってます！乗ります


In [14]:
# csvファイルとして保存
all_wakati_df.to_csv('all_text_wakatigaki_181201.csv', index=False, encoding='utf-8')

In [15]:
# pklファイルとして保存
all_wakati_df.to_pickle('all_text_wakatigaki_181201.pkl')